# SQL Project
You were hired by Ironhack to perform an Analytics Consulting Project entitled: competitive landscape.

Your mission is to create and populate an appropriate database with many coding schools that are our competition, as well as design an suitable queries that answer business questions of interest (to be defined by you)


**Suggested Steps in the Project:**


*   Read this notebook and understand each function. Comment the code appropriately

*   Populate the list of schools with a wider variety of schools (how are you going to get the school ID?)

* Take a look at the obtained dataframes. What dimensions do you have? what keys do you have? how could the different dataframes be connected?

* Go back to the drawing board and try to create an entity relationship diagram for tables available

* Once you have the schemas you want, you will need to:
  - create the suitable SQL queries to create the tables and populate them
  - run these queries using the appropriate Python connectors
  
* Bonus: How will this datamodel be updated in the future? Please write auxiliary functions that test the database for data quality issues. For example: how could you make sure you only include the most recent comments when you re-run the script?


# Suggested Deliverables

* 5-6 minute presentation of data model created, decision process and business analysis proposed

* exported .sql file with the final schema

* Supporting python files used to generate all logic

* High level documentation explaining tables designed and focusing on update methods

Crucial hint: check out the following tutorial:
https://www.dataquest.io/blog/sql-insert-tutorial/


1. Get Data for Comments

In [63]:
# you must populate this dict with the schools required -> try talking to the teaching team about this
# adding another 4 competitors as they are ranked high on the list from different bootcamps Ironhack offers

schools = {   
'ironhack' : 10828,
'app-academy' : 10525,
'springboard' : 11035,
'designlab':10697,
'evolve-security-academy':11584,
'syntax-technologies':11797,
'clarusway': 11539
}

import re
import pandas as pd
from pandas.io.json import json_normalize
import requests



def get_comments_school(school):
    TAG_RE = re.compile(r'<[^>]+>')
    # defines url to make api call to data -> dynamic with school if you want to scrape competition
    url = "https://www.switchup.org/chimera/v1/school-review-list?mainTemplate=school-review-list&path=%2Fbootcamps%2F" + school + "&isDataTarget=false&page=3&perPage=10000&simpleHtml=true&truncationLength=250"
    #makes get request and converts answer to json
    # url defines the page of all the information, request is made, and information is returned to data variable
    data = requests.get(url).json()
    #converts json to dataframe
    reviews =  pd.DataFrame(data['content']['reviews'])
  
    #aux function to apply regex and remove tags
    def remove_tags(x):
        return TAG_RE.sub('',x)
    reviews['review_body'] = reviews['body'].apply(remove_tags)
    reviews['school'] = school
    return reviews

In [ ]:
#dataframe reviews has another two columns 'review_body' and 'school'

In [103]:
comments = []

for school in schools.keys():
    print(school)
    comments.append(get_comments_school(school))

comments = pd.concat(comments)
#comments are the reviews dataframe in the function get_comments_school(school)

ironhack
app-academy
springboard
designlab
evolve-security-academy
syntax-technologies
clarusway


In [104]:
#to show all the columns
pd.set_option('display.max.columns', None)
comments.head()

,id,name,anonymous,hostProgramName,graduatingYear,isAlumni,jobTitle,tagline,body,rawBody,createdAt,queryDate,program,user,overallScore,comments,overall,curriculum,jobSupport,review_body,school
0,306372,Sergio Burgos,False,None,2023.0,False,International Negotiator,The Most Intense Academic Challenge,"<span class=""truncatable""><p></p><p>After comp...",<p>After completing my Data Analytics Bootcamp...,11/10/2023,2023-11-10,Data Analytics Bootcamp,{'image': None},3.3,[],3.0,3.0,4.0,After completing my Data Analytics Bootcamp wi...,ironhack
1,306215,Anonymous,True,None,2023.0,True,,Transformative Experience: My Time at Ironhack,"<span class=""truncatable""><p></p><p>Pros: 1)In...",<p>Pros: 1)Intensive Learning 2)Real-World Pro...,11/6/2023,2023-11-06,Web Development Bootcamp,{'image': None},4.0,[],4.0,4.0,4.0,Pros: 1)Intensive Learning 2)Real-World Projec...,ironhack
2,306068,Anonymous,True,None,2023.0,False,Full stack development,Now I can do it,"<span class=""truncatable""><p></p><p>7 months a...","<p>7 months ago, I only had an idea about html...",10/31/2023,2023-10-31,None,{'image': None},5.0,[],5.0,5.0,5.0,"7 months ago, I only had an idea about html an...",ironhack
3,305297,Utku Cikmaz,False,None,2023.0,False,Full Stack Web Developer,It was good,"<span class=""truncatable""><p></p><p>The course...","<p>The course was great. Especially, Luis is a...",10/2/2023,2023-10-02,Web Development Bootcamp,{'image': None},4.0,[],5.0,3.0,4.0,"The course was great. Especially, Luis is a gr...",ironhack
4,305278,Nirmal Hodge,False,None,2023.0,False,Product Designer,Ironhack 100% Worth It!,"<span class=""truncatable""><p></p><p>I joined t...",<p>I joined the UX/ UI Bootcamp and to be hone...,9/30/2023,2023-09-30,UX/UI Design Bootcamp,{'image': None},5.0,[],5.0,5.0,5.0,I joined the UX/ UI Bootcamp and to be honest ...,ironhack


2. Data Cleaning for Comments

In [105]:
comments.shape


(5532, 21)

In [106]:
comments.dtypes

id                   int64
name                object
anonymous             bool
hostProgramName     object
graduatingYear     float64
isAlumni              bool
jobTitle            object
tagline             object
body                object
rawBody             object
createdAt           object
queryDate           object
program             object
user                object
overallScore        object
comments            object
overall             object
curriculum          object
jobSupport          object
review_body         object
school              object
dtype: object

In [107]:
#among all the columns, we can already drop body as it is the column with tags, same with rawBody
#name is not important to our analysis
#anonymous is not important
#user (with image) can alos be dropped
#createdAt and queryDate have different format, needs to be changed
#data type of graduatingYear can be converted into int
all_comments = comments.drop(columns=['body', 'rawBody', 'name', 'anonymous', 'user'])
all_comments['queryDate'] = pd.to_datetime(all_comments['queryDate']).dt.strftime('%d/%m/%Y')

In [108]:
all_comments['graduatingYear'] = all_comments['graduatingYear'].astype('Int64')

In [109]:
#the comment columns looks fishy
all_comments['comments'].value_counts()
#Ah okay, the column is filled with answers from the school to the feedback. Then we can also drop

[]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

In [110]:
all_comments.drop(columns=['comments'], inplace=True)


In [111]:
all_comments.head()

,id,hostProgramName,graduatingYear,isAlumni,jobTitle,tagline,createdAt,queryDate,program,overallScore,overall,curriculum,jobSupport,review_body,school
0,306372,None,2023,False,International Negotiator,The Most Intense Academic Challenge,11/10/2023,10/11/2023,Data Analytics Bootcamp,3.3,3.0,3.0,4.0,After completing my Data Analytics Bootcamp wi...,ironhack
1,306215,None,2023,True,,Transformative Experience: My Time at Ironhack,11/6/2023,06/11/2023,Web Development Bootcamp,4.0,4.0,4.0,4.0,Pros: 1)Intensive Learning 2)Real-World Projec...,ironhack
2,306068,None,2023,False,Full stack development,Now I can do it,10/31/2023,31/10/2023,None,5.0,5.0,5.0,5.0,"7 months ago, I only had an idea about html an...",ironhack
3,305297,None,2023,False,Full Stack Web Developer,It was good,10/2/2023,02/10/2023,Web Development Bootcamp,4.0,5.0,3.0,4.0,"The course was great. Especially, Luis is a gr...",ironhack
4,305278,None,2023,False,Product Designer,Ironhack 100% Worth It!,9/30/2023,30/09/2023,UX/UI Design Bootcamp,5.0,5.0,5.0,5.0,I joined the UX/ UI Bootcamp and to be honest ...,ironhack


In [112]:
all_comments['hostProgramName'].value_counts()
#It seems that the None record is actually missing value NA

Software Engineering    978
UX/UI Design            583
Data Science            265
Web Development          82
Cybersecurity            78
Data Analytics           37
Digital Marketing        24
Bootcamp Prep            14
Name: hostProgramName, dtype: int64

In [113]:
all_comments['hostProgramName'].isnull().value_counts()
#there are 3471 missing values, we can see if column program does a better job to indicate the bootcamp

True     3471
False    2061
Name: hostProgramName, dtype: int64

In [114]:
all_comments['program'].isnull().value_counts()
#the records with data definitely higher than hostProgramName

False    5239
True      293
Name: program, dtype: int64

In [115]:
all_comments['program'].isnull().value_counts()

False    5239
True      293
Name: program, dtype: int64

In [116]:
all_comments['program'].value_counts()

Software Engineer Track: In-Person                                                                  794
UX Academy Foundations                                                                              620
Full-time Web Development Bootcamp                                                                  362
UI/UX Design Career Track                                                                           314
Data Science Career Track                                                                           306
                                                                                                   ... 
Cyber Security                                                                                        1
Full Stack Web Development with IT Fundamentals - Premium Full Time/Part Time with Limited Seats      1
Data Analytics                                                                                        1
Data Science with Prep                                          

In [117]:
#Before I remove one of the two columns, I want to see if the rows with missing value in column program actually have info in hostProgramName
col_comp = all_comments.loc[all_comments['program'].isnull() & ~all_comments['hostProgramName'].isnull()]
col_comp


,id,hostProgramName,graduatingYear,isAlumni,jobTitle,tagline,createdAt,queryDate,program,overallScore,overall,curriculum,jobSupport,review_body,school
618,256206,UX/UI Design,2018,True,None,Revamped my portfolio,10/8/2019,08/10/2019,None,4.3,4.0,4.0,5.0,I enrolled in the full-time UX/UI design progr...,ironhack
626,255372,Software Engineering,2019,True,None,Ironhack opened up a new world for me - it can...,9/27/2019,27/09/2019,None,5.0,5.0,5.0,5.0,I joined Ironhack because I wanted to change m...,ironhack
627,255316,UX/UI Design,2019,True,None,Great for the right audience,9/26/2019,26/09/2019,None,3.0,3.0,3.0,NaN,I came across Ironhack's program while hunting...,ironhack
630,254398,Software Engineering,2018,True,None,"Part time web dev, great way to learn JS &amp;...",9/11/2019,11/09/2019,None,5.0,5.0,5.0,5.0,I won't lie learning web dev was a tough. Ther...,ironhack
631,254352,Software Engineering,2019,True,None,Best coding bootcamp ever!,9/11/2019,11/09/2019,None,5.0,5.0,5.0,5.0,"Hi there, I know that it is a big decision and...",ironhack
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
802,254410,UX/UI Design,2019,True,None,Great course! The material was very helpful!,9/11/2019,11/09/2019,None,4.7,5.0,4.0,5.0,The content was very informative and easy to g...,designlab
806,252788,UX/UI Design,2018,True,None,Loved it!,8/6/2019,06/08/2019,None,5.0,5.0,5.0,5.0,The course flow was exactly what I needed to m...,designlab
170,254376,Cybersecurity,2019,True,None,Life/Career Changing Experience,9/11/2019,11/09/2019,None,5.0,5.0,5.0,5.0,I started this cybersecurity Bootcamp not havi...,evolve-security-academy
171,253747,Cybersecurity,2018,True,None,Game changing for your career; Timely conte...,8/26/2019,26/08/2019,None,5.0,5.0,5.0,5.0,I highly recommend Evolve to enhance cyber sec...,evolve-security-academy


In [118]:
for i in col_comp.index:
    all_comments.loc[i, 'program'] = all_comments.loc[i, 'hostProgramName']

all_comments

,id,hostProgramName,graduatingYear,isAlumni,jobTitle,tagline,createdAt,queryDate,program,overallScore,overall,curriculum,jobSupport,review_body,school
0,306372,None,2023,False,International Negotiator,The Most Intense Academic Challenge,11/10/2023,10/11/2023,Data Analytics Bootcamp,3.3,3.0,3.0,4.0,After completing my Data Analytics Bootcamp wi...,ironhack
1,306215,None,2023,True,,Transformative Experience: My Time at Ironhack,11/6/2023,06/11/2023,Web Development Bootcamp,4.0,4.0,4.0,4.0,Pros: 1)Intensive Learning 2)Real-World Projec...,ironhack
2,306068,None,2023,False,Full stack development,Now I can do it,10/31/2023,31/10/2023,None,5.0,5.0,5.0,5.0,"7 months ago, I only had an idea about html an...",ironhack
3,305297,None,2023,False,Full Stack Web Developer,It was good,10/2/2023,02/10/2023,Web Development Bootcamp,4.0,5.0,3.0,4.0,"The course was great. Especially, Luis is a gr...",ironhack
4,305278,None,2023,False,Product Designer,Ironhack 100% Worth It!,9/30/2023,30/09/2023,UX/UI Design Bootcamp,5.0,5.0,5.0,5.0,I joined the UX/ UI Bootcamp and to be honest ...,ironhack
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,275858,None,2021,False,AWS|DEVOPS ENGINEER,"I'm so glad to know you, Clarusway",4/14/2021,14/04/2021,AWS Solutions Architect/DevOps Engineer for Be...,5.0,5.0,5.0,5.0,I have worked in a different industry for year...,clarusway
236,275854,None,2021,False,Full Stack Developer,Thanks to Clarusway!,4/14/2021,14/04/2021,Full Stack Development for Beginners,5.0,5.0,5.0,5.0,The Clarusway Full Stack Developer program is ...,clarusway
237,275849,None,2021,False,Jr. DevOps Engineer,Make your IT dream job come true.,4/14/2021,14/04/2021,AWS Solutions Architect/DevOps Engineer for Be...,5.0,5.0,5.0,5.0,The bootcamp of Clarusway is a compressed educ...,clarusway
238,275820,None,2021,False,Cloud Architect,I’m glad and feel lucky that I decided to beco...,4/14/2021,14/04/2021,AWS Solutions Architect/DevOps Engineer for Be...,5.0,5.0,5.0,5.0,I am 40 years old. Before I join this bootcamp...,clarusway


In [100]:
#based on the result, we can see that the bootcamp name is more generalized in hostProgramName, and this column indeed compensates the column program (86 rows)
#therefore, we can extract the generalized names in program and merge them in hostProgramName
#bootcamp = ['Software Engineering', 'UX/UI Design', 'Data Science', 'Web Development','Cybersecurity', 'Data Analytics', 'Digital Marketing', 'Bootcamp Prep']



In [125]:
#this part is slightly changed with the definition of UX/UI, and Web Development

def map_program_category(program):
   if 'Data Analytics' in str(program):
       return 'Data Analytics Bootcamp'
   elif 'UX' in str(program):
       return 'UX/UI Design Bootcamp'
   elif 'UI' in str(program):
       return 'UX/UI Design Bootcamp'
   elif 'Web Development' in str(program):
       return 'Web Development Bootcamp'
   elif 'Web Design' in str(program):
       return 'Web Development Bootcamp'
   elif 'Cybersecurity' in str(program):
       return 'Cybersecurity Bootcamp'
   else:
       return 'Other'

all_comments['program_category'] = comments['program'].apply(map_program_category)



In [126]:
all_comments['program_category'].value_counts()

Other                       2588
UX/UI Design Bootcamp       1653
Web Development Bootcamp     871
Data Analytics Bootcamp      326
Cybersecurity Bootcamp        94
Name: program_category, dtype: int64

In [128]:
#Now we can drop both program and hostProgramName
all_comments.drop(columns=['program', 'hostProgramName'], inplace=True)

In [129]:
all_comments.head()
#Now I only have 14 columns left. Not sure about jobTitle and the individual scores yet as overallScore may already be enough if we do not dive deep.But I'll keep them for now.

,id,graduatingYear,isAlumni,jobTitle,tagline,createdAt,queryDate,overallScore,overall,curriculum,jobSupport,review_body,school,program_category
0,306372,2023,False,International Negotiator,The Most Intense Academic Challenge,11/10/2023,10/11/2023,3.3,3.0,3.0,4.0,After completing my Data Analytics Bootcamp wi...,ironhack,Data Analytics Bootcamp
1,306215,2023,True,,Transformative Experience: My Time at Ironhack,11/6/2023,06/11/2023,4.0,4.0,4.0,4.0,Pros: 1)Intensive Learning 2)Real-World Projec...,ironhack,Web Development Bootcamp
2,306068,2023,False,Full stack development,Now I can do it,10/31/2023,31/10/2023,5.0,5.0,5.0,5.0,"7 months ago, I only had an idea about html an...",ironhack,Other
3,305297,2023,False,Full Stack Web Developer,It was good,10/2/2023,02/10/2023,4.0,5.0,3.0,4.0,"The course was great. Especially, Luis is a gr...",ironhack,Web Development Bootcamp
4,305278,2023,False,Product Designer,Ironhack 100% Worth It!,9/30/2023,30/09/2023,5.0,5.0,5.0,5.0,I joined the UX/ UI Bootcamp and to be honest ...,ironhack,UX/UI Design Bootcamp


In [24]:
all_comments.duplicated().value_counts()
#No duplicates are found

False    5532
dtype: int64

In [132]:
#We still need to change the columns(overallScore, overall, curriculum, jobSupport ) into data type float for EDA 
all_comments[['overallScore', 'overall', 'curriculum', 'jobSupport']] =all_comments[['overallScore', 'overall', 'curriculum', 'jobSupport']].astype(float)
all_comments.dtypes

id                    int64
graduatingYear        Int64
isAlumni               bool
jobTitle             object
tagline              object
createdAt            object
queryDate            object
overallScore        float64
overall             float64
curriculum          float64
jobSupport          float64
review_body          object
school               object
program_category     object
dtype: object

In [137]:
#Job Titles is a bit messy, let's create a function to clean up and harmonize job titles
#and replace the NaN values with 'No info'
all_comments['jobTitle'].fillna('No info', inplace=True)
# Your cleaning function for job titles
def clean_job_title(title):
    # Replace 'none' or 'looking for a job' with 'No info'
    title = re.sub(r'(?i)\b(?:none|looking for a job)\b', 'No info', title)
    
    # Additional cleaning logic for job titles harmonization
    title = re.sub(r'(?i)\b(?:fullstack|full stack|full-stack)\b', 'Full Stack', title)
    title = re.sub(r'(?i)\b(?:ux/ui|ux/ui design|ux/ui designer)\b', 'UX/UI Designer', title)
    title = re.sub(r'(?i)\b(?:software developer|software eng|developer|dev)\b', 'Software Developer', title)
    title = re.sub(r'(?i)\b(?:data analyst|data analytics)\b', 'Data Analyst', title)
    title = re.sub(r'(?i)\b(?:cybersecurity|cyber security|info sec)\b', 'Cybersecurity Analyst', title)
    
    # Remove non-alphanumeric characters
    title = re.sub(r'[^a-zA-Z0-9\s]', '', title)
    
    # Remove extra whitespaces
    title = re.sub(r'\s+', ' ', title).strip()
    
    # Replace empty spaces with 'No info'
    title = 'No info' if title == '' else title
    
    # Convert to uppercase
    title = title.upper()
    
    return title

all_comments['jobTitle'] = all_comments['jobTitle'].apply(clean_job_title)

# Display the cleaned DataFrame
all_comments.head()


,id,graduatingYear,isAlumni,jobTitle,tagline,createdAt,queryDate,overallScore,overall,curriculum,jobSupport,review_body,school,program_category
0,306372,2023,False,INTERNATIONAL NEGOTIATOR,The Most Intense Academic Challenge,11/10/2023,10/11/2023,3.3,3.0,3.0,4.0,After completing my Data Analytics Bootcamp wi...,ironhack,Data Analytics Bootcamp
1,306215,2023,True,NO INFO,Transformative Experience: My Time at Ironhack,11/6/2023,06/11/2023,4.0,4.0,4.0,4.0,Pros: 1)Intensive Learning 2)Real-World Projec...,ironhack,Web Development Bootcamp
2,306068,2023,False,FULL STACK DEVELOPMENT,Now I can do it,10/31/2023,31/10/2023,5.0,5.0,5.0,5.0,"7 months ago, I only had an idea about html an...",ironhack,Other
3,305297,2023,False,FULL STACK WEB SOFTWARE DEVELOPER,It was good,10/2/2023,02/10/2023,4.0,5.0,3.0,4.0,"The course was great. Especially, Luis is a gr...",ironhack,Web Development Bootcamp
4,305278,2023,False,PRODUCT DESIGNER,Ironhack 100% Worth It!,9/30/2023,30/09/2023,5.0,5.0,5.0,5.0,I joined the UX/ UI Bootcamp and to be honest ...,ironhack,UX/UI Design Bootcamp


In [156]:
#Unify the column names, and give overallScore a different name to differentiate it from overall
col_comm = ['id', 'graduate_year', 'is_alumni', 'job_title', 'tagline', 'create_date', 'query_date', 'total_score', 'overall', 'curriculum', 'job_support', 'review_body', 'school', 'program_category']
all_comments.columns = col_comm

3. Get Data for Schools

In [138]:
from pandas.io.json import json_normalize

def get_school_info(school, school_id):
    url = 'https://www.switchup.org/chimera/v1/bootcamp-data?mainTemplate=bootcamp-data%2Fdescription&path=%2Fbootcamps%2F'+ str(school) + '&isDataTarget=false&bootcampId='+ str(school_id) + '&logoTag=logo&truncationLength=250&readMoreOmission=...&readMoreText=Read%20More&readLessText=Read%20Less'

    data = requests.get(url).json()

    data.keys()

    courses = data['content']['courses']
    courses_df = pd.DataFrame(courses, columns= ['courses'])

    locations = data['content']['locations']
    locations_df = json_normalize(locations)

    badges_df = pd.DataFrame(data['content']['meritBadges'])
    
    website = data['content']['webaddr']
    description = data['content']['description']
    logoUrl = data['content']['logoUrl']
    school_df = pd.DataFrame([website,description,logoUrl]).T
    school_df.columns =  ['website','description','LogoUrl']

    locations_df['school'] = school
    courses_df['school'] = school
    badges_df['school'] = school
    school_df['school'] = school
    

    locations_df['school_id'] = school_id
    courses_df['school_id'] = school_id
    badges_df['school_id'] = school_id
    school_df['school_id'] = school_id

    return locations_df, courses_df, badges_df, school_df

locations_list = []
courses_list = []
badges_list = []
schools_list = []

for school, id in schools.items():
    print(school)
    a,b,c,d = get_school_info(school,id)
    
    locations_list.append(a)
    courses_list.append(b)
    badges_list.append(c)
    schools_list.append(d)



ironhack


C:\Users\ruoxi\AppData\Local\Temp\ipykernel_23148\1181667300.py:14: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  locations_df = json_normalize(locations)


app-academy


C:\Users\ruoxi\AppData\Local\Temp\ipykernel_23148\1181667300.py:14: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  locations_df = json_normalize(locations)


springboard


...

C:\Users\ruoxi\AppData\Local\Temp\ipykernel_23148\1181667300.py:14: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  locations_df = json_normalize(locations)


syntax-technologies


C:\Users\ruoxi\AppData\Local\Temp\ipykernel_23148\1181667300.py:14: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  locations_df = json_normalize(locations)


clarusway


C:\Users\ruoxi\AppData\Local\Temp\ipykernel_23148\1181667300.py:14: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  locations_df = json_normalize(locations)


In [140]:
locations = pd.concat(locations_list)
locations

,id,description,country.id,country.name,country.abbrev,city.id,city.name,city.keyword,state.id,state.name,state.abbrev,state.keyword,school,school_id
0,15901,"Berlin, Germany",57.0,Germany,DE,31156.0,Berlin,berlin,NaN,NaN,NaN,NaN,ironhack,10828
1,16022,"Mexico City, Mexico",29.0,Mexico,MX,31175.0,Mexico City,mexico-city,NaN,NaN,NaN,NaN,ironhack,10828
2,16086,"Amsterdam, Netherlands",59.0,Netherlands,NL,31168.0,Amsterdam,amsterdam,NaN,NaN,NaN,NaN,ironhack,10828
3,16088,"Sao Paulo, Brazil",42.0,Brazil,BR,31121.0,Sao Paulo,sao-paulo,NaN,NaN,NaN,NaN,ironhack,10828
4,16109,"Paris, France",38.0,France,FR,31136.0,Paris,paris,NaN,NaN,NaN,NaN,ironhack,10828
5,16375,"Miami, FL, United States",1.0,United States,US,31.0,Miami,miami,11.0,Florida,FL,florida,ironhack,10828
6,16376,"Madrid, Spain",12.0,Spain,ES,31052.0,Madrid,madrid,NaN,NaN,NaN,NaN,ironhack,10828
7,16377,"Barcelona, Spain",12.0,Spain,ES,31170.0,Barcelona,barcelona,NaN,NaN,NaN,NaN,ironhack,10828
8,16709,"Lisbon, Portugal",28.0,Portugal,PT,31075.0,Lisbon,lisbon,NaN,NaN,NaN,NaN,ironhack,10828
9,17233,Online,NaN,NaN,NaN,NaN,NaN,NaN,1.0,Online,Online,online,ironhack,10828


4. Data cleaning for School Locations

In [144]:
locations.dtypes
#country.id,  city.id, state.id should be int64, however, since the country.id and city id are not globaly used, I would consider them as columns to be deleted
#country.abbrev gave the same information as country.name, and I will keep country.name because it's more elaberate
#it's the same with city.name and city.keyword, and [state.id	state.name	state.abbrev	state.keyword]
#description actually gives more information about whether it's online or not
#the column name can be unified by replacing '.' with '_'

id                  int64
description        object
country.id        float64
country.name       object
country.abbrev     object
city.id           float64
city.name          object
city.keyword       object
state.id          float64
state.name         object
state.abbrev       object
state.keyword      object
school             object
school_id           int64
dtype: object

In [145]:
locations.duplicated().value_counts()

False    18
dtype: int64

In [146]:
all_locations = locations.drop(columns=['country.id',  'city.id', 'state.id', 'city.keyword','country.abbrev', 'state.abbrev', 'state.keyword'])

In [147]:
col_loc = ['id', 'description', 'country_name', 'city_name', 'state_name', 'school', 'school_id']
all_locations.columns = col_loc
all_locations

,id,description,country_name,city_name,state_name,school,school_id
0,15901,"Berlin, Germany",Germany,Berlin,NaN,ironhack,10828
1,16022,"Mexico City, Mexico",Mexico,Mexico City,NaN,ironhack,10828
2,16086,"Amsterdam, Netherlands",Netherlands,Amsterdam,NaN,ironhack,10828
3,16088,"Sao Paulo, Brazil",Brazil,Sao Paulo,NaN,ironhack,10828
4,16109,"Paris, France",France,Paris,NaN,ironhack,10828
5,16375,"Miami, FL, United States",United States,Miami,Florida,ironhack,10828
6,16376,"Madrid, Spain",Spain,Madrid,NaN,ironhack,10828
7,16377,"Barcelona, Spain",Spain,Barcelona,NaN,ironhack,10828
8,16709,"Lisbon, Portugal",Portugal,Lisbon,NaN,ironhack,10828
9,17233,Online,NaN,NaN,Online,ironhack,10828


In [148]:
#Since the state does not really have a lot of records and also does not give enough information, I'll delete it.
#Meanwhile the missing values from country_name and city_name will be replaced by "Online", so that all the information from description is populated in country_name and city_name
all_locations['country_name'].fillna('Online', inplace=True)
all_locations['city_name'].fillna('Online', inplace=True)
all_locations.drop(columns = ['state_name', 'description'], inplace = True)
all_locations


,id,country_name,city_name,school,school_id
0,15901,Germany,Berlin,ironhack,10828
1,16022,Mexico,Mexico City,ironhack,10828
2,16086,Netherlands,Amsterdam,ironhack,10828
3,16088,Brazil,Sao Paulo,ironhack,10828
4,16109,France,Paris,ironhack,10828
5,16375,United States,Miami,ironhack,10828
6,16376,Spain,Madrid,ironhack,10828
7,16377,Spain,Barcelona,ironhack,10828
8,16709,Portugal,Lisbon,ironhack,10828
9,17233,Online,Online,ironhack,10828


In [149]:
courses = pd.concat(courses_list)
courses

,courses,school,school_id
0,Cyber Security Bootcamp,ironhack,10828
1,Cybersecurity Part-Time,ironhack,10828
2,Data Analytics Bootcamp,ironhack,10828
3,Data Analytics Part-Time,ironhack,10828
4,UX/UI Design Bootcamp,ironhack,10828
...,...,...,...
26,React JS Development,clarusway,11539
27,Salesforce Administrator & Consultant,clarusway,11539
28,Salesforce Administrator & Developer,clarusway,11539
29,Salesforce Advanced Flow Course,clarusway,11539


5. Data cleaning for School Courses

In [150]:
courses.dtypes

courses      object
school       object
school_id     int64
dtype: object

In [152]:
courses.duplicated().value_counts()
#There's not much to be cleaned

False    69
dtype: int64

In [142]:
badges = pd.concat(badges_list)
badges.head()

,name,keyword,description,school,school_id
0,Available Online,available_online,<p>School offers fully online courses</p>,ironhack,10828
1,Verified Outcomes,verified_outcomes,<p>School publishes a third-party verified out...,ironhack,10828
2,Flexible Classes,flexible_classes,<p>School offers part-time and evening classes...,ironhack,10828
0,Available Online,available_online,<p>School offers fully online courses</p>,app-academy,10525
1,Flexible Classes,flexible_classes,<p>School offers part-time and evening classes...,app-academy,10525


In [154]:
# any data cleaning still missing here? take a look at the description
schools = pd.concat(schools_list)
schools

,website,description,LogoUrl,school,school_id
0,www.ironhack.com/en,"<span class=""truncatable""><p>Ironhack is a glo...",https://d92mrp7hetgfk.cloudfront.net/images/si...,ironhack,10828
0,appacademy.io,"<span class=""truncatable""><p>Founded in 2012, ...",https://d92mrp7hetgfk.cloudfront.net/images/si...,app-academy,10525
0,www.springboard.com/?utm_source=switchup&utm_m...,"<span class=""truncatable""><p>Springboard is an...",https://d92mrp7hetgfk.cloudfront.net/images/si...,springboard,11035
0,designlab.com,"<span class=""truncatable""><p>Designlab teaches...",https://d92mrp7hetgfk.cloudfront.net/images/si...,designlab,10697
0,//www.mentodesign.academy/,"<span class=""truncatable""><p>Mento Design Acad...",https://d92mrp7hetgfk.cloudfront.net/images/si...,evolve-security-academy,11584
0,www.syntaxtechs.com/,"<span class=""truncatable""><p>Syntax Technologi...",https://d92mrp7hetgfk.cloudfront.net/images/si...,syntax-technologies,11797
0,clarusway.com/,"<span class=""truncatable""><p>Clarusway is a hi...",https://d92mrp7hetgfk.cloudfront.net/images/si...,clarusway,11539


In [155]:
#unify the names of the columns
col_sch = ['website', 'description', 'logo_url', 'school', 'school_id']
schools.columns = col_sch

In [157]:
#I want to see how the description looks like
schools.iloc[0, 1]

'<span class="truncatable"><p>Ironhack is a global tech school with 9 campuses worldwide, located in Miami, Madrid, Barcelona, Paris, Berlin, Amsterdam, Mexico City, Lisbon and São Paulo. Ironhack offers bootcamps and part-time courses in Web Development, UX/UI Design, Data Analytics<span class="read-more-text">... <a class="read-more-button">Read More</a></span><span class="read-more" style="display:none;"> and Cyber Security, which are taught both remotely and in-person.</span></p><p>With more than 6.000 graduates working at companies like Google, Visa, Twitter, Rocket Internet and Orange, among others, Ironhack has an extensive global network of +600 partner companies. Post graduation, all students have access to career services, which prepare graduates for their job searches and facilitate interviews in their city\'s local tech ecosystem.</p><span class=\'read-less-text\' style=\'display:none;\'><a class=\'read-less-button\'>Read Less</a></span></span>'

In [158]:
def remove_tags(x):
        return re.compile(r'<[^>]+>').sub('',x)

schools['new_description'] = schools['description'].apply(remove_tags)


In [160]:
schools.drop(columns = ['description'], inplace=True)

In [161]:
schools

,website,logo_url,school,school_id,new_description
0,www.ironhack.com/en,https://d92mrp7hetgfk.cloudfront.net/images/si...,ironhack,10828,Ironhack is a global tech school with 9 campus...
0,appacademy.io,https://d92mrp7hetgfk.cloudfront.net/images/si...,app-academy,10525,"Founded in 2012, App Academy is a world-renown..."
0,www.springboard.com/?utm_source=switchup&utm_m...,https://d92mrp7hetgfk.cloudfront.net/images/si...,springboard,11035,Springboard is an online learning platform tha...
0,designlab.com,https://d92mrp7hetgfk.cloudfront.net/images/si...,designlab,10697,Designlab teaches in-demand UX/UI design skill...
0,//www.mentodesign.academy/,https://d92mrp7hetgfk.cloudfront.net/images/si...,evolve-security-academy,11584,Mento Design Academy is a UX &amp; UI Bootcamp...
0,www.syntaxtechs.com/,https://d92mrp7hetgfk.cloudfront.net/images/si...,syntax-technologies,11797,Syntax Technologies is the world’s leading onl...
0,clarusway.com/,https://d92mrp7hetgfk.cloudfront.net/images/si...,clarusway,11539,Clarusway is a highly respected IT training sc...
